# Process PA DOC testing data from emails

* We received the daily PA DOC testing data from a bunch of forwarded emails.


* Some of them contained pasted tables (presumably from an Excel file) while others were an Excel attachment.


* __ISSUES__
  1. the format and column names and some data changes over the period from April 6 through July 
  2. The email tables were difficult to scrape because of HTML format issues etc.
  

* __PROCEDURE__
  1. we compiled four different tyoes of excel files for the four different formats we identified.
  2. the email tables where pasted into a sheet/tab of a spreadsheet with the date as the sheet name
  3. This notebook processes these four different formats and does the appropriate column selection and renaming
  4. The appropriate report date is added for each row extracted and a single data frame results

----------

#### HISTORY

* 11/20/20 mbod - add in the inmate testing purpose columns
* 7/21/20 mbod - parse the four different spreadsheets into a single dataframe

### Setup

In [321]:
import pandas as pd
import os
import re

In [322]:
DATA_DIR = '../data/PA_DOC_testing_data/'

In [323]:
files_to_process=[f for f in os.listdir(DATA_DIR) if f.endswith('.xlsx')]

In [324]:
files_to_process.sort()
files_to_process

['April6-June1.xlsx',
 'June2-8.xlsx',
 'PA-DOC-COVID-19-Testing_10-1.xlsx',
 'PA-DOC-COVID-19-Testing_10-10.xlsx',
 'PA-DOC-COVID-19-Testing_10-11.xlsx',
 'PA-DOC-COVID-19-Testing_10-12.xlsx',
 'PA-DOC-COVID-19-Testing_10-13.xlsx',
 'PA-DOC-COVID-19-Testing_10-14.xlsx',
 'PA-DOC-COVID-19-Testing_10-15.xlsx',
 'PA-DOC-COVID-19-Testing_10-16.xlsx',
 'PA-DOC-COVID-19-Testing_10-17.xlsx',
 'PA-DOC-COVID-19-Testing_10-18.xlsx',
 'PA-DOC-COVID-19-Testing_10-19.xlsx',
 'PA-DOC-COVID-19-Testing_10-2.xlsx',
 'PA-DOC-COVID-19-Testing_10-20.xlsx',
 'PA-DOC-COVID-19-Testing_10-21.xlsx',
 'PA-DOC-COVID-19-Testing_10-22.xlsx',
 'PA-DOC-COVID-19-Testing_10-23.xlsx',
 'PA-DOC-COVID-19-Testing_10-24.xlsx',
 'PA-DOC-COVID-19-Testing_10-25.xlsx',
 'PA-DOC-COVID-19-Testing_10-26.xlsx',
 'PA-DOC-COVID-19-Testing_10-31.xlsx',
 'PA-DOC-COVID-19-Testing_10-5.xlsx',
 'PA-DOC-COVID-19-Testing_10-6.xlsx',
 'PA-DOC-COVID-19-Testing_10-7.xlsx',
 'PA-DOC-COVID-19-Testing_10-8.xlsx',
 'PA-DOC-COVID-19-Testing_10-9.

### 1. Processing `Staff_Turn_Away` reports for 6-10 through 7-15


![](img/staff_turn_around_excel1.png)

#### Columns and rows to extract

* First 25 rows after headers and `Central Office` have the SCI institutions of interest
    * Down to `Waymart`
    * Ignore `Training Academy` and `Parole` and the other facilities/locations in blue
    
    
* Keep these columns:
    1. `SCI/Institution` map to `SCI`
    2. `Staff Positive` map to 'staff_positive`
    3. `Staff Negative` map to 'staff_negative`
    4. `Staff Pending` map to 'staff_pending`
    5. `Staff Death With(+) Test` map to `staff_death`
    6. `Staff Recovered` map to `incarcerated_person_recovered`
    7. `Inmate Positive` map to `incarcerated_person_positive`
    8. `Inmate Negative` map to `incarcerated_person_negative`
    9. `Inmate Pending` map to `incarcerated_person_pending`
    10. `Inmate Death With(+) Test` map to `incarcerated_person_death`
    11. `Inmate Recovered` map to `incarcerated_person_recovered`    


* Add date column

In [325]:
column_mapping = {
 'SCI/Institution': 'SCI',
 'Staff Positive': 'staff_positive',
 'Staff Negative': 'staff_negative',
 'Staff Pending': 'staff_pending',
 'Staff Death With(+) Test ': 'staff_death',
 'Staff Recovered': 'staff_recovered',
 'Inmate Positive': 'incarcerated_person_positive',
 'Inmate Negative': 'incarcerated_person_negative',
 'Inmate Pending': 'incarcerated_person_pending',
 'Inmate Death With(+) Test ': 'incarcerated_person_death',
 'Inmate Recovered': 'incarcerated_person_recovered',
 # testing purpose columns
 'Transfer': 'test_transfer', 
 'Transfer (+)': 'test_transfer_positive', 
 'Release': 'test_release',
 'Release (+)': 'test_release_positive', 
 'Hospital': 'test_hospital', 
 'Hospital (+)': 'test_hospital_positive', 
 'Surveilance': 'test_surveilance',
 'Surveilance (+)': 'test_surveilance_positive', 
 'Symptomatic': 'test_symptomatic', 
 'Symptomatic (+)': 'test_symptomatic_positive', 
 'Miscellaneous': 'test_miscellaneous'
}

In [326]:
test_df = pd.read_excel(os.path.join(DATA_DIR, 'Staff_Turn_Away_Report_6-10.xlsx'))

In [327]:
select_columns=test_df.columns[:]

In [328]:
test_df2=test_df.iloc[1:26][select_columns]
test_df2.rename(columns=column_mapping, inplace=True)
test_df2['date']='2020-06-10'

In [329]:
test_df2.head()

,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,incarcerated_person_death,...,test_release,test_release_positive,test_hospital,test_hospital_positive,test_surveilance,test_surveilance_positive,test_symptomatic,test_symptomatic_positive,test_miscellaneous,date
1,Albion,NaN,7.0,4.0,NaN,NaN,NaN,62.0,NaN,NaN,...,48.0,NaN,3.0,NaN,NaN,NaN,1.0,NaN,NaN,2020-06-10
2,Benner Township,NaN,11.0,1.0,NaN,NaN,1.0,82.0,NaN,NaN,...,39.0,1.0,3.0,NaN,36.0,NaN,5.0,NaN,NaN,2020-06-10
3,Cambridge Springs,NaN,7.0,2.0,NaN,NaN,NaN,126.0,NaN,NaN,...,116.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2020-06-10
4,Camp Hill,4.0,8.0,20.0,NaN,3.0,1.0,985.0,2.0,NaN,...,104.0,NaN,6.0,NaN,NaN,NaN,7.0,1.0,NaN,2020-06-10
5,Chester,16.0,14.0,17.0,NaN,13.0,5.0,231.0,NaN,NaN,...,72.0,NaN,2.0,NaN,147.0,3.0,3.0,2.0,NaN,2020-06-10


In [330]:
def process_STA_report(fname):
    
    month, day = re.match('Staff_Turn_Away_Report_(\d+)-(\d+)\.xlsx', fname).groups()
    
    date = "2020-{:0>2}-{:0>2}".format(month,day)
    
    df = pd.read_excel(os.path.join(DATA_DIR, fname))
    
    df2=df.iloc[1:26][select_columns]
    df2.rename(columns=column_mapping, inplace=True)
    df2['date']=date
    
    return df2

In [331]:
data_type1 = []
for dsheet in files_to_process:
    if dsheet.startswith('Staff_Turn_Away_Report_'):
        print('Processing', dsheet)
        data_type1.append(process_STA_report(dsheet))

Processing Staff_Turn_Away_Report_6-10.xlsx
Processing Staff_Turn_Away_Report_6-11.xlsx
Processing Staff_Turn_Away_Report_6-12.xlsx
Processing Staff_Turn_Away_Report_6-15.xlsx
Processing Staff_Turn_Away_Report_6-16.xlsx
Processing Staff_Turn_Away_Report_6-18.xlsx
Processing Staff_Turn_Away_Report_6-22.xlsx
Processing Staff_Turn_Away_Report_6-23.xlsx
Processing Staff_Turn_Away_Report_6-24.xlsx
Processing Staff_Turn_Away_Report_6-25.xlsx
Processing Staff_Turn_Away_Report_6-26.xlsx
Processing Staff_Turn_Away_Report_6-29.xlsx
Processing Staff_Turn_Away_Report_6-30.xlsx
Processing Staff_Turn_Away_Report_7-1.xlsx
Processing Staff_Turn_Away_Report_7-10.xlsx
Processing Staff_Turn_Away_Report_7-13.xlsx
Processing Staff_Turn_Away_Report_7-14.xlsx
Processing Staff_Turn_Away_Report_7-15.xlsx
Processing Staff_Turn_Away_Report_7-2.xlsx
Processing Staff_Turn_Away_Report_7-6.xlsx
Processing Staff_Turn_Away_Report_7-7.xlsx
Processing Staff_Turn_Away_Report_7-8.xlsx
Processing Staff_Turn_Away_Report_7-9

### 2. PA DOC Covid Testing data sheet

![](img/covid_testing_excel.png)

#### Columns and rows to extract

* Skip first row on load to get column headings

* First 25 rows after headers and `Central Office` have the SCI institutions of interest
    * Down to `Waymart`
    * Ignore `Training Academy` and `Parole` and the other facilities/locations in blue
    
    
* Keep these columns:
    1. `LOCATION` map to `SCI`
    2. `Positive` (col 2) map to 'staff_positive`
    3. `Negative` (col 3) map to 'staff_negative`
    4. `Pending` (col 4) map to 'staff_pending`
    5. `Death` map to `staff_death`
    6. `Recovered` map to `incarcerated_person_recovered`
    7. `Positive` (col 7) map to 'incarcerated_person_positive`
    8. `Negative` map to 'incarcerated_person_negative`
    9. `Pending` map to 'incarcerated_person_pending`
    10. `Death` map to `incarcerated_person_death`
    11. `Recovered` map to `incarcerated_person_recovered`    


* Add date column

In [332]:
column_mapping2 = {
 'Unnamed: 0': 'SCI',
 'Positive': 'staff_positive',
 'Negative': 'staff_negative',
 'Pending': 'staff_pending',
 'Death*': 'staff_death',
 'Recovered': 'staff_recovered',
 'Positive.1': 'incarcerated_person_positive',
 'Negative.1': 'incarcerated_person_negative',
 'Pending.1': 'incarcerated_person_pending',
 'Death*.1': 'incarcerated_person_death',
 'Recovered.1': 'incarcerated_person_recovered',
    
  # test purpose columns added 
  # 'Transfer', 'Transfer (+)', 'Release', 'Release (+)', 'Hospital',
  # 'Hospital (+)', 'Surveilance', 'Surveilance (+)', 'Symptomatic',
  # 'Symptomatic (+)'

 'Transfer': 'test_transfer', 
 'Transfer (+)': 'test_transfer_positive', 
 'Release': 'test_release',
 'Release (+)': 'test_release_positive', 
 'Hospital': 'test_hospital', 
 'Hospital (+)': 'test_hospital_positive', 
 'Surveilance': 'test_surveilance',
 'Surveilance (+)': 'test_surveilance_positive', 
 'Symptomatic': 'test_symptomatic', 
 'Symptomatic (+)': 'test_symptomatic_positive' 
    
}

In [333]:
def process_testing_report(fname):
    
    month, day = re.match('PA-DOC-COVID-19-Testing_(\d+)-(\d+)\.xlsx', fname).groups()
    
    date = "2020-{:0>2}-{:0>2}".format(month,day)
    
    df = pd.read_excel(os.path.join(DATA_DIR, fname), skiprows=1)
    df.rename(columns=column_mapping2, inplace=True)
    df2=df.iloc[1:26][column_mapping2.values()]
    df2['date']=date
    
    return df2

In [334]:
process_testing_report('PA-DOC-COVID-19-Testing_7-19.xlsx')

,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,incarcerated_person_death,...,test_transfer_positive,test_release,test_release_positive,test_hospital,test_hospital_positive,test_surveilance,test_surveilance_positive,test_symptomatic,test_symptomatic_positive,date
1,Albion,NaN,11.0,4.0,NaN,NaN,3.0,134.0,NaN,NaN,...,1.0,112.0,2.0,9.0,NaN,1.0,NaN,3.0,NaN,2020-07-19
2,Benner Township,NaN,17.0,6.0,NaN,NaN,1.0,155.0,NaN,1.0,...,NaN,79.0,1.0,10.0,NaN,37.0,NaN,6.0,NaN,2020-07-19
3,Cambridge Springs,NaN,10.0,1.0,NaN,NaN,2.0,197.0,NaN,NaN,...,NaN,177.0,2.0,1.0,NaN,6.0,NaN,2.0,NaN,2020-07-19
4,Camp Hill,4.0,8.0,29.0,NaN,3.0,1.0,1779.0,1.0,NaN,...,NaN,216.0,NaN,8.0,NaN,NaN,NaN,8.0,1.0,2020-07-19
5,Chester,16.0,14.0,18.0,NaN,14.0,5.0,299.0,NaN,NaN,...,NaN,140.0,NaN,4.0,NaN,147.0,3.0,6.0,2.0,2020-07-19
6,Coal Township,2.0,17.0,1.0,NaN,2.0,4.0,240.0,NaN,NaN,...,NaN,101.0,NaN,8.0,NaN,74.0,4.0,5.0,NaN,2020-07-19
7,Dallas,3.0,11.0,3.0,NaN,NaN,NaN,202.0,NaN,NaN,...,NaN,92.0,NaN,7.0,NaN,1.0,NaN,6.0,NaN,2020-07-19
8,Fayette,9.0,40.0,4.0,NaN,5.0,1.0,184.0,NaN,NaN,...,NaN,130.0,NaN,6.0,1.0,NaN,NaN,4.0,NaN,2020-07-19
9,Forest,NaN,9.0,3.0,NaN,NaN,NaN,106.0,NaN,NaN,...,NaN,66.0,NaN,3.0,NaN,NaN,NaN,4.0,NaN,2020-07-19
10,Frackville,NaN,20.0,11.0,NaN,NaN,NaN,112.0,NaN,NaN,...,NaN,96.0,NaN,2.0,NaN,NaN,NaN,2.0,NaN,2020-07-19


In [335]:
tdf = pd.read_excel('../data/PA_DOC_testing_data/PA-DOC-COVID-19-Testing_7-19.xlsx', skiprows=1)

In [336]:
tdf.rename(columns=column_mapping2)[1:26]

,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,incarcerated_person_death,...,test_hospital_positive,test_surveilance,test_surveilance_positive,test_symptomatic,test_symptomatic_positive,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25
1,Albion,NaN,11.0,4.0,NaN,NaN,3.0,134.0,NaN,NaN,...,NaN,1.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
2,Benner Township,NaN,17.0,6.0,NaN,NaN,1.0,155.0,NaN,1.0,...,NaN,37.0,NaN,6.0,NaN,NaN,The positive column shows number of positive t...,NaN,NaN,NaN
3,Cambridge Springs,NaN,10.0,1.0,NaN,NaN,2.0,197.0,NaN,NaN,...,NaN,6.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
4,Camp Hill,4.0,8.0,29.0,NaN,3.0,1.0,1779.0,1.0,NaN,...,NaN,NaN,NaN,8.0,1.0,NaN,NaN,NaN,NaN,NaN
5,Chester,16.0,14.0,18.0,NaN,14.0,5.0,299.0,NaN,NaN,...,NaN,147.0,3.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN
6,Coal Township,2.0,17.0,1.0,NaN,2.0,4.0,240.0,NaN,NaN,...,NaN,74.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN
7,Dallas,3.0,11.0,3.0,NaN,NaN,NaN,202.0,NaN,NaN,...,NaN,1.0,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN
8,Fayette,9.0,40.0,4.0,NaN,5.0,1.0,184.0,NaN,NaN,...,1.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
9,Forest,NaN,9.0,3.0,NaN,NaN,NaN,106.0,NaN,NaN,...,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN
10,Frackville,NaN,20.0,11.0,NaN,NaN,NaN,112.0,NaN,NaN,...,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [337]:
data_type2 = []
for dsheet in files_to_process:
    if dsheet.startswith('PA-DOC-COVID-19-Testing_'):
        print('Processing', dsheet)
        data_type2.append(process_testing_report(dsheet))

Processing PA-DOC-COVID-19-Testing_10-1.xlsx
Processing PA-DOC-COVID-19-Testing_10-10.xlsx
Processing PA-DOC-COVID-19-Testing_10-11.xlsx
Processing PA-DOC-COVID-19-Testing_10-12.xlsx
Processing PA-DOC-COVID-19-Testing_10-13.xlsx
Processing PA-DOC-COVID-19-Testing_10-14.xlsx
Processing PA-DOC-COVID-19-Testing_10-15.xlsx
Processing PA-DOC-COVID-19-Testing_10-16.xlsx
Processing PA-DOC-COVID-19-Testing_10-17.xlsx
Processing PA-DOC-COVID-19-Testing_10-18.xlsx
Processing PA-DOC-COVID-19-Testing_10-19.xlsx
Processing PA-DOC-COVID-19-Testing_10-2.xlsx
Processing PA-DOC-COVID-19-Testing_10-20.xlsx
Processing PA-DOC-COVID-19-Testing_10-21.xlsx
Processing PA-DOC-COVID-19-Testing_10-22.xlsx
Processing PA-DOC-COVID-19-Testing_10-23.xlsx
Processing PA-DOC-COVID-19-Testing_10-24.xlsx
Processing PA-DOC-COVID-19-Testing_10-25.xlsx
Processing PA-DOC-COVID-19-Testing_10-26.xlsx
Processing PA-DOC-COVID-19-Testing_10-31.xlsx
Processing PA-DOC-COVID-19-Testing_10-5.xlsx
Processing PA-DOC-COVID-19-Testing_10

### 3. Multiple tab spreadsheet for June 2-8

In [338]:
ss = pd.ExcelFile('../data/PA_DOC_testing_data/June2-8.xlsx')

In [339]:
ss.sheet_names

['6-8', '6-5', '6-4', '6-2']

In [340]:
ss.parse('6-4')

,SCI/Institution,Staff Positive,Staff Negative,Staff Pending,Staff Death With(+) Test,Staff Recovered,Inmate Positive,Inmate Negative,Inmate Pending,Inmate Death With(+) Test,Inmate Recovered,Transfer,Release,Hospital,Surveilance,Symptomatic,Miscellaneous
0,Central Office,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Albion,NaN,6,5.0,NaN,NaN,NaN,52.0,1.0,NaN,NaN,3.0,47.0,2.0,NaN,1.0,NaN
2,Benner Township,NaN,11,NaN,NaN,NaN,1.0,80.0,NaN,NaN,NaN,NaN,34.0,3.0,36.0,5.0,3.0
3,Cambridge Springs,NaN,7,1.0,NaN,NaN,NaN,111.0,NaN,NaN,NaN,6.0,96.0,NaN,NaN,2.0,7.0
4,Camp Hill,4.0,8,19.0,NaN,3.0,1.0,803.0,93.0,NaN,NaN,793.0,84.0,6.0,NaN,7.0,7.0
5,Chester,16.0,14,15.0,NaN,13.0,5.0,217.0,NaN,NaN,2.0,NaN,68.0,2.0,147.0,3.0,2.0
6,Coal Township,1.0,12,1.0,NaN,1.0,NaN,53.0,NaN,NaN,NaN,6.0,40.0,NaN,NaN,4.0,3.0
7,Dallas,2.0,11,3.0,NaN,NaN,NaN,71.0,NaN,NaN,NaN,6.0,43.0,5.0,NaN,3.0,14.0
8,Fayette,4.0,34,2.0,NaN,2.0,1.0,55.0,NaN,NaN,NaN,2.0,40.0,5.0,NaN,1.0,8.0
9,Forest,NaN,5,1.0,NaN,NaN,NaN,27.0,NaN,NaN,NaN,5.0,18.0,1.0,NaN,3.0,NaN


In [341]:
column_mapping3 = dict([(k.strip(),v) for k,v in column_mapping.items()])
column_mapping3

{'SCI/Institution': 'SCI',
 'Staff Positive': 'staff_positive',
 'Staff Negative': 'staff_negative',
 'Staff Pending': 'staff_pending',
 'Staff Death With(+) Test': 'staff_death',
 'Staff Recovered': 'staff_recovered',
 'Inmate Positive': 'incarcerated_person_positive',
 'Inmate Negative': 'incarcerated_person_negative',
 'Inmate Pending': 'incarcerated_person_pending',
 'Inmate Death With(+) Test': 'incarcerated_person_death',
 'Inmate Recovered': 'incarcerated_person_recovered',
 'Transfer': 'test_transfer',
 'Transfer (+)': 'test_transfer_positive',
 'Release': 'test_release',
 'Release (+)': 'test_release_positive',
 'Hospital': 'test_hospital',
 'Hospital (+)': 'test_hospital_positive',
 'Surveilance': 'test_surveilance',
 'Surveilance (+)': 'test_surveilance_positive',
 'Symptomatic': 'test_symptomatic',
 'Symptomatic (+)': 'test_symptomatic_positive',
 'Miscellaneous': 'test_miscellaneous'}

In [342]:
data_type3=[]
for sname in ss.sheet_names:
    print('Processing',sname)
    month, day = sname.split('-')
    date = "2020-{:0>2}-{:0>2}".format(month,day)
    df=ss.parse(sname)
    df.rename(columns=column_mapping3, inplace=True)
    try:
        df2=df[column_mapping3.values()]
    except:
        df2=df
    df2['date']=date
    
    # drop Central Office
    df3 = df2[df2['SCI']!='Central Office']
    data_type3.append(df3)

Processing 6-8
Processing 6-5
Processing 6-4
Processing 6-2


### 4. Multiple tab spreadsheet for April 6 - June 1

In [343]:
ss2 = pd.ExcelFile('../data/PA_DOC_testing_data/April6-June1.xlsx')

In [344]:
ss2.sheet_names

['6-1',
 '5-29',
 '5-28',
 '5-27',
 '5-26',
 '5-22',
 '5-21',
 '5-20',
 '5-19',
 '5-15',
 '5-14',
 '5-13',
 '5-12',
 '5-11',
 '5-8',
 '5-7',
 '5-6',
 '5-5',
 '4-10',
 '4-9',
 '4-8',
 '4-7']

In [345]:
cnames=set()
for sn in ss2.sheet_names:
    cnames.update(list(ss2.parse(sn, skiprows=1).columns.values))

In [346]:
cnames

{'COVID TESTING',
 'Death with (+) Test',
 'Hospital',
 'Miscellaneous',
 'Negative',
 'Negative.1',
 'Other',
 'Pending',
 'Pending.1',
 'Positive',
 'Positive.1',
 'Recovered',
 'Release',
 'Surveilance',
 'Symptomatic',
 'Total',
 'Total Tests by site',
 'Total.1',
 'Transfer',
 'Unnamed: 6'}

In [347]:
column_mapping4 = {
 'COVID TESTING': 'SCI',
 'Positive': 'staff_positive',
 'Negative': 'staff_negative',
 'Pending': 'staff_pending',
 'Positive.1': 'incarcerated_person_positive',
 'Negative.1': 'incarcerated_person_negative',
 'Pending.1': 'incarcerated_person_pending',
 'Death with (+) Test': 'incarcerated_person_death',
 'Recovered': 'incarcerated_person_recovered',

  # add test purpose columns
  # 'Transfer', 'Release', 'Hospital', 'Surveilance', 'Symptomatic', 'Miscellaneous'
'Transfer': 'test_transfer',
 'Release': 'test_release',
 'Hospital': 'test_hospital',
 'Surveilance': 'test_surveilance',
 'Symptomatic': 'test_symptomatic',
 'Miscellaneous': 'test_miscellaneous'
    
}

In [348]:
d=ss2.parse('5-21',skiprows=1).rename(columns=column_mapping4)

In [349]:
d.columns[d.columns.isin(column_mapping4.values())]

Index(['SCI', 'staff_positive', 'staff_negative', 'staff_pending',
       'incarcerated_person_positive', 'incarcerated_person_negative',
       'incarcerated_person_pending', 'incarcerated_person_death',
       'incarcerated_person_recovered'],
      dtype='object')

In [350]:
data_type4=[]
for sname in ss2.sheet_names:
    print('processing',sname)
    month, day = sname.split('-')
    date = "2020-{:0>2}-{:0>2}".format(month,day)
    df=ss2.parse(sname, skiprows=1)
    df.rename(columns=column_mapping4, inplace=True)
    cols_to_keep = df.columns[df.columns.isin(column_mapping4.values())]
    df2=df[cols_to_keep]
    df2['date']=date
    
    # drop Central Office
    df3 = df2[df2['SCI']!='Central Office']
    
    data_type4.append(df3)

processing 6-1
processing 5-29
processing 5-28
processing 5-27
processing 5-26
processing 5-22
processing 5-21
processing 5-20
processing 5-19
processing 5-15
processing 5-14
processing 5-13
processing 5-12
processing 5-11
processing 5-8
processing 5-7
processing 5-6
processing 5-5
processing 4-10
processing 4-9
processing 4-8
processing 4-7


/Users/mattodonnell/files/PA-SCI_COVID19/venv/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


### Putting it all together

In [351]:
all_data=pd.concat(data_type1 + data_type2 + data_type3 + data_type4)

In [352]:
all_data.head()

,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,incarcerated_person_death,...,test_release,test_release_positive,test_hospital,test_hospital_positive,test_surveilance,test_surveilance_positive,test_symptomatic,test_symptomatic_positive,test_miscellaneous,date
1,Albion,NaN,7.0,4.0,NaN,NaN,NaN,62.0,NaN,NaN,...,48.0,NaN,3.0,NaN,NaN,NaN,1.0,NaN,NaN,2020-06-10
2,Benner Township,NaN,11.0,1.0,NaN,NaN,1.0,82.0,NaN,NaN,...,39.0,1.0,3.0,NaN,36.0,NaN,5.0,NaN,NaN,2020-06-10
3,Cambridge Springs,NaN,7.0,2.0,NaN,NaN,NaN,126.0,NaN,NaN,...,116.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,2020-06-10
4,Camp Hill,4.0,8.0,20.0,NaN,3.0,1.0,985.0,2.0,NaN,...,104.0,NaN,6.0,NaN,NaN,NaN,7.0,1.0,NaN,2020-06-10
5,Chester,16.0,14.0,17.0,NaN,13.0,5.0,231.0,NaN,NaN,...,72.0,NaN,2.0,NaN,147.0,3.0,3.0,2.0,NaN,2020-06-10


In [353]:
all_data['date'].value_counts()

2020-07-13    25
2020-10-06    25
2020-11-18    25
2020-06-18    25
2020-06-29    25
              ..
2020-05-29    14
2020-05-28    14
2020-06-01    13
2020-06-05    12
2020-06-08    12
Name: date, Length: 98, dtype: int64

In [354]:
all_data['SCI'].value_counts()

Chester              98
Dallas               98
Cambridge Springs    98
Benner Township      98
Houtzdale            98
Forest               98
Camp Hill            98
Fayette              98
Greene               98
Frackville           98
Coal Township        98
Albion               98
Huntingdon           96
Laurel Highlands     95
Mahanoy              93
Mercer               93
Phoenix              91
Pine Grove           91
Quehanna BC          91
Muncy                91
Retreat              76
Rockview             74
Waymart              72
Somerset             72
Smithfield           72
Name: SCI, dtype: int64

In [355]:
all_data.index=pd.DatetimeIndex(all_data['date'])
all_data.sort_index(inplace=True)

In [356]:
all_data['date'].describe()

count           2283
unique            98
top       2020-07-13
freq              25
Name: date, dtype: object

In [357]:
all_data

,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,incarcerated_person_death,...,test_release,test_release_positive,test_hospital,test_hospital_positive,test_surveilance,test_surveilance_positive,test_symptomatic,test_symptomatic_positive,test_miscellaneous,date
date,,,,,,,,,,,,,,,,,,,,,
2020-04-07,Rockview,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-07
2020-04-07,Albion,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-07
2020-04-07,Benner Township,NaN,4.0,1.0,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-07
2020-04-07,Cambridge Springs,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-07
2020-04-07,Camp Hill,2.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-04-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-23,Rockview,53.0,41.0,20.0,NaN,12.0,46.0,763.0,93.0,1.0,...,266.0,1.0,32.0,4.0,25.0,13.0,36.0,23.0,NaN,2020-11-23
2020-11-23,Smithfield,39.0,130.0,20.0,NaN,18.0,38.0,508.0,153.0,NaN,...,245.0,5.0,36.0,5.0,96.0,16.0,14.0,10.0,NaN,2020-11-23
2020-11-23,Somerset,41.0,47.0,22.0,NaN,10.0,210.0,640.0,475.0,NaN,...,320.0,4.0,121.0,4.0,466.0,179.0,32.0,19.0,NaN,2020-11-23


In [358]:
doc2_df = all_data.copy()
doc2_df=doc2_df.drop(columns='date').reset_index()

exclude_cols = ['SCI', 'date', 'date.1']

cols_to_use = [c for c in doc2_df.columns if c not in exclude_cols]

for col in cols_to_use:
    print('Calculating delta for', col)
    doc2_df[f'{col}_new'] = doc2_df.groupby('SCI')[col].diff()
    
doc2_df=doc2_df.set_index(doc2_df['date'])

Calculating delta for staff_positive
Calculating delta for staff_negative
Calculating delta for staff_pending
Calculating delta for staff_death
Calculating delta for staff_recovered
Calculating delta for incarcerated_person_positive
Calculating delta for incarcerated_person_negative
Calculating delta for incarcerated_person_pending
Calculating delta for incarcerated_person_death
Calculating delta for incarcerated_person_recovered
Calculating delta for test_transfer
Calculating delta for test_transfer_positive
Calculating delta for test_release
Calculating delta for test_release_positive
Calculating delta for test_hospital
Calculating delta for test_hospital_positive
Calculating delta for test_surveilance
Calculating delta for test_surveilance_positive
Calculating delta for test_symptomatic
Calculating delta for test_symptomatic_positive
Calculating delta for test_miscellaneous


In [471]:
doc2_df.to_csv('../data/latest_data/PA_DOC_testing_data.csv')

In [360]:
doc2_df

,date,SCI,staff_positive,staff_negative,staff_pending,staff_death,staff_recovered,incarcerated_person_positive,incarcerated_person_negative,incarcerated_person_pending,...,test_transfer_positive_new,test_release_new,test_release_positive_new,test_hospital_new,test_hospital_positive_new,test_surveilance_new,test_surveilance_positive_new,test_symptomatic_new,test_symptomatic_positive_new,test_miscellaneous_new
date,,,,,,,,,,,,,,,,,,,,,
2020-04-07,2020-04-07,Rockview,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Albion,NaN,5.0,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Benner Township,NaN,4.0,1.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Cambridge Springs,NaN,NaN,2.0,NaN,NaN,NaN,2.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-07,2020-04-07,Camp Hill,2.0,1.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-23,2020-11-23,Rockview,53.0,41.0,20.0,NaN,12.0,46.0,763.0,93.0,...,0.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,NaN
2020-11-23,2020-11-23,Smithfield,39.0,130.0,20.0,NaN,18.0,38.0,508.0,153.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2020-11-23,2020-11-23,Somerset,41.0,47.0,22.0,NaN,10.0,210.0,640.0,475.0,...,0.0,0.0,0.0,0.0,0.0,1.0,123.0,0.0,2.0,NaN
